In [16]:
import os
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

from mcmc import initialize_slab, spin_flip, spin_flip_canonical, change_site, get_random_idx
from catkit.gen.adsorption import get_adsorption_sites

from ase.calculators.eam import EAM
from ase.calculators.lammpsrun import LAMMPS

import numpy as np

## For semi-grand canonical

In [17]:
# Cu alat from https://www.copper.org/resources/properties/atomic_properties.html
Cu_alat = 3.6147
slab = initialize_slab(Cu_alat)

# get ALL the adsorption sites
# top should have connectivity 1, bridge should be 2 and hollow more like 4
coords, connectivity, sym_idx = get_adsorption_sites(slab, symmetry_reduced=False)
print(f"In pristine slab, there are a total of {len(connectivity)} sites")

# state of each vacancy in slab. for state > 0, it's filled, and that's the index of the adsorbate atom in slab
state = np.zeros(len(coords), dtype=int)

temp = 300
pot = 2

In pristine slab, there are a total of 64 sites


In [18]:
# test top
top_site = get_random_idx(connectivity, type="top")
bridge_site = get_random_idx(connectivity, type="bridge")
hollow_site = get_random_idx(connectivity, type="hollow")

# add to 3 sites
spin_flip(state, slab, temp, pot, coords, connectivity, save_cif=True, iter=1, site_idx=top_site, testing=True, folder_name="different_sites_noalat")
spin_flip(state, slab, temp, pot, coords, connectivity, save_cif=True, iter=2, site_idx=bridge_site, testing=True, folder_name="different_sites_noalat")
spin_flip(state, slab, temp, pot, coords, connectivity, save_cif=True, iter=3, site_idx=hollow_site, testing=True, folder_name="different_sites_noalat")

DEBUG:
 we are at iter 1
DEBUG:idx is 3 with connectivity 1 at [7.66793665e+00 1.61421949e-16 2.04220500e+01]
DEBUG:before proposed state is
DEBUG:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
DEBUG:chosen site is empty
DEBUG:adsorbing from empty to adsorbate Cu and potential 2
DEBUG:current slab has 64 atoms
DEBUG:proposed slab has 65 atoms
DEBUG:after proposed state is
DEBUG:[ 0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
DEBUG:
 we are at iter 2
DEBUG:idx is 30 with connectivity 2 at [ 5.11195776  8.94592609 20.42205   ]
DEBUG:before proposed state is
DEBUG:[ 0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
DEBU

(array([ 0,  0,  0, 64,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 65,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 66,  0]),
 Gratoms(symbols='Cu67', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surface_atoms=..., tags=...),
 0,
 False)

In [19]:
# Test cases
import logging
logging.basicConfig(
    format="%(levelname)s:%(message)s",
    level=logging.DEBUG,
    datefmt="%m/%d/%Y %I:%M:%S %p",
)

site_idx = top_site
# start_ads = slab[state[site_idx]].symbol
# print(f"start ads {start_ads}")
adsorbates = ['In', 'Cu', 'Ga']
pots = [2, 3, 5]

# try top already filled site
slab, state, delta_pot, start_ads, end_ads = change_site(slab, state, pots, adsorbates, coords, site_idx)
print("Trial move")
print(f"slab={slab},state={state},delta_pot={delta_pot},start_ads={start_ads},end_ads={end_ads}")

slab, state, delta_pot, start_ads, end_ads = change_site(slab, state, pots, adsorbates, coords, site_idx, start_ads=end_ads, end_ads=start_ads)
print("Revert move")
print(f"slab={slab},state={state},delta_pot={delta_pot},start_ads={start_ads},end_ads={end_ads}")


# adsorb one more random site
site_idx = len(state) - 1
slab, state, delta_pot, start_ads, end_ads = change_site(slab, state, pots, adsorbates, coords, site_idx)
print("Revert move")
print(f"slab={slab},state={state},delta_pot={delta_pot},start_ads={start_ads},end_ads={end_ads}")

# ads_pot_dict = dict(zip(adsorbates, pots))

# ads_choices = adsorbates.copy()
# ads_choices.remove(start_ads)
# print(f"ads choices are {ads_choices}")

# print(f"{ad.remove("")}")

DEBUG:chosen site already adsorbed
DEBUG:chosen ads is In
DEBUG:current slab has 67 atoms
DEBUG:replacing Cu with In
DEBUG:proposed slab has 67 atoms
DEBUG:chosen site already adsorbed
DEBUG:chosen ads is Cu
DEBUG:current slab has 67 atoms
DEBUG:replacing In with Cu
DEBUG:proposed slab has 67 atoms
DEBUG:chosen site is empty
DEBUG:adsorbing from empty to adsorbate Cu and potential 3
DEBUG:current slab has 67 atoms
DEBUG:proposed slab has 68 atoms


Trial move
slab=Gratoms(symbols='Cu66In', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surface_atoms=..., tags=...),state=[ 0  0  0 66  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 65  0],delta_pot=-1,start_ads=Cu,end_ads=In
Revert move
slab=Gratoms(symbols='Cu67', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surface_atoms=..., tags=...),state=[ 0  0  0 66  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 65  0],delta_pot=1,start_ads=In,end_ads=Cu
Revert move
slab=Gratoms(symbols='Cu68', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surfa

In [20]:
# Possible test cases, can go in sequence
# 1) empty list
# 2) start with ads, go to empty
# 3) start wtih ads, go to another ads

In [21]:
# Other test cases for spin flip

## For canonical

In [22]:
import itertools
adsorbed_idx = np.argwhere(state != 0).flatten()
curr_ads = {k: list(g) for k, g in itertools.groupby(adsorbed_idx, key=lambda x: slab[state[x]].symbol)}


In [23]:
curr_ads

{'Cu': [3, 30, 62, 63]}

In [24]:
empty_idx = np.argwhere(state == 0).flatten().tolist()
curr_ads['None'] = empty_idx

In [25]:
import random
type1, type2 = random.sample(curr_ads.keys(), 2)
site1_idx, site2_idx = [random.choice(curr_ads[x]) for x in [type1, type2]]

Spin flip canonical

In [32]:
state, slab, energy, accept = spin_flip_canonical(state, slab, temp, coords, connectivity, iter=1, testing=True, adsorbates=['Cu', 'Ga', 'None'])

DEBUG:current ads {'Cu': [3, 15, 17, 30], 'None': [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]}
DEBUG:
 we are at iter 1
DEBUG:idx is 3 with connectivity 1 at [7.66793665e+00 1.61421949e-16 2.04220500e+01]
DEBUG:idx is 61 with connectivity 4 at [ 6.3899472   3.83396832 20.42205   ]
DEBUG:before proposed state is
DEBUG:[ 0  0  0 64  0  0  0  0  0  0  0  0  0  0  0 66  0 65  0  0  0  0  0  0
  0  0  0  0  0  0 67  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
DEBUG:current slab has 68 atoms
DEBUG:chosen site already adsorbed
DEBUG:chosen ads is None
DEBUG:current slab has 68 atoms
DEBUG:desorbing Cu
DEBUG:proposed slab has 67 atoms
DEBUG:chosen site is empty
DEBUG:adsorbing from empty to adsorbate Cu and potential 0
DEBUG:current slab has 67 atoms
DEB